In [1]:
import xarray as xr
import json
from pandas import to_datetime, IntervalIndex
from pathlib import Path
from numpy import uint16, float32

from util import date_interval_endpoints as endpoints
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY 

home = str(Path.home())

In [2]:
cube = xr.open_zarr(home+"/stacks/13/2002/8920.zarr", chunks=None)["data"]

timestamps = [to_datetime(v['timestamp']) for v in json.loads(cube.json).values()]
cube.coords['slice_ids'] = timestamps
cube = cube.rename({'slice_ids': 'timestamps'})
cube = cube.sortby('timestamps')
cube = cube.assign_coords(bands=[b.split('_')[0] for b in cube.bands.data])
cube = xr.Dataset({b: cube.sel(dict(bands=b)).reset_coords('bands', drop=True) for b in cube.bands.data})
cube

<xarray.Dataset>
Dimensions:     (timestamps: 35, x: 256, y: 256)
Coordinates:
  * timestamps  (timestamps) datetime64[ns] 2019-06-02T09:58:14 ... 2019-08-29T10:07:49
Dimensions without coordinates: x, y
Data variables:
    B04         (timestamps, x, y) uint16 291 296 294 286 ... 4121 4117 4109
    B03         (timestamps, x, y) uint16 621 646 672 668 ... 4601 4599 4601
    B02         (timestamps, x, y) uint16 390 394 396 392 ... 5129 5137 5143
    B08         (timestamps, x, y) uint16 5122 5304 5430 5331 ... 5074 5093 5116

In [3]:
starttime = datetime(*min(timestamps).timetuple()[:3],0,0,0)
endtime = datetime(*max(timestamps).timetuple()[:3],0,0,0)

eps = endpoints(starttime, endtime, 16)
idx = IntervalIndex.from_arrays(eps[::2], eps[1::2])
idx

IntervalIndex([(2019-06-02, 2019-06-15 23:59:59], (2019-06-16, 2019-06-30 23:59:59], (2019-07-01, 2019-07-15 23:59:59], (2019-07-16, 2019-07-31 23:59:59], (2019-08-01, 2019-08-15 23:59:59], (2019-08-16, 2019-08-29 23:59:59]],
              closed='right',
              dtype='interval[datetime64[ns]]')

In [4]:
# Calculate values in time intervals
avg_cube = cube.groupby_bins('timestamps',bins=idx).mean('timestamps')

# Fake data
from numpy.random import random_sample
for b in ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12']:
    avg_cube[b] = avg_cube.B04
    avg_cube[b].data = random_sample(avg_cube.B04.shape)*4000

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) float64 1.812e+03 ... 3.099e+03
    B03              (timestamps_bins, x, y) float64 2.067e+03 ... 3.353e+03
    B02              (timestamps_bins, x, y) float64 2.029e+03 ... 3.623e+03
    B08              (timestamps_bins, x, y) float64 4.83e+03 ... 4.542e+03
    B05              (timestamps_bins, x, y) float64 2.377e+03 ... 2.392e+03
    B06              (timestamps_bins, x, y) float64 2.542e+03 ... 2.464e+03
    B07              (timestamps_bins, x, y) float64 1.808e+03 360.8 ... 915.1
    B8A              (timestamps_bins, x, y) float64 3.884e+03 ... 319.2
    B11              (timestamps_bins, x, y) float64 2.271e+03 ... 165.0
    B12              (timestamps_bins, x, y) float64 799.5 1.492e+03 ... 546.8

In [5]:
def calculate_index(a,b):
    return ((a-b)/(a+b)+1)/2 # stretch [-1,+1] to [0,1]

ics = { # index components
  "NDVI":  ["B08", "B04"],
  "GNDVI": ["B08", "B03"],
  "BNDVI": ["B08", "B02"],
  "NDSI":  ["B11", "B12"],
  "NDWI":  ["B03", "B08"]
}

for ic in ics:
    avg_cube[ic] = calculate_index(avg_cube[ics[ic][0]], avg_cube[ics[ic][1]])*65335 # 2**16

# Typing conforming to SH Mass output
avg_cube = avg_cube.astype(uint16)
avg_cube['CVI'] = (avg_cube.B08 * avg_cube.B05 / avg_cube.B03**2).astype(float32)

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) uint16 1811 1859 1911 ... 3182 3099
    B03              (timestamps_bins, x, y) uint16 2067 2119 2167 ... 3389 3353
    B02              (timestamps_bins, x, y) uint16 2029 2066 2107 ... 3599 3622
    B08              (timestamps_bins, x, y) uint16 4829 4985 5074 ... 4569 4541
    B05              (timestamps_bins, x, y) uint16 2376 2401 238 ... 2619 2391
    B06              (timestamps_bins, x, y) uint16 2541 3408 2606 ... 2164 2463
    B07              (timestamps_bins, x, y) uint16 1808 360 485 ... 2245 915
    B8A              (timestamps_bins, x, y) uint16 3883 3806 2575 ... 3639 319
    B11              (timestamps_bins, x, y) uint16 2271 3698 880 ... 1321 164
    B12              (timestamps_bins, x, y) uint16 799 1491 1996 ... 805 546
    NDVI             (timestamps_bins, x, y) uint16 47513 47588 ... 38514 38835
    GNDVI            (timestamps_bins, x, y) uint16 45752 45847 ... 37511 37585
    BNDVI            (timestamps_bins, x, y) uint16 46004 46186 ... 36546 36343
    NDSI             (timestamps_bins, x, y) uint16 48324 46557 ... 40584 15145
    NDWI             (timestamps_bins, x, y) uint16 19582 19487 ... 27823 27749
    CVI              (timestamps_bins, x, y) float32 0.38769862 ... 1.226351